# Title

TODO: write up about project

Notes on installation?
https://github.com/stefan-jansen/machine-learning-for-trading/tree/main/installation

### Import libraries

In [30]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import zipline

%load_ext zipline
%matplotlib inline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


### Download stock data

In [2]:
start_date='2011-01-01'
end_date='2021-11-06'

In [11]:
def download_stock_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    df = df.reset_index()
    df = df.drop(['Adj Close'], axis=1)
    df = df.rename(str.lower, axis='columns')
    df['dividend'] = 0
    df['split'] = 1
    return df

In [12]:
df_tsla = download_stock_data('TSLA', start_date, end_date)
df_aapl = download_stock_data('AAPL', start_date, end_date)
df_nflx = download_stock_data('NFLX', start_date, end_date)
df_hsbc = download_stock_data('HSBC', start_date, end_date)
df_gs = download_stock_data('GS', start_date, end_date)
df_gme = download_stock_data('GME', start_date, end_date)
df_ge = download_stock_data('GE', start_date, end_date)
df_lyg = download_stock_data('LYG', start_date, end_date)
df_shldq = download_stock_data('SHLDQ', start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Save data to CSV file
Note: the CSV file must be the ticker symbol without additional text like tsla-2011.csv

In [ ]:
df_tsla.to_csv('daily/tsla.csv', index=False)
df_aapl.to_csv('daily/aapl.csv', index=False)
df_nflx.to_csv('daily/nflx.csv', index=False)
df_hsbc.to_csv('daily/hsbc.csv', index=False)
df_gs.to_csv('daily/gs.csv', index=False)
df_gme.to_csv('daily/gme.csv', index=False)
df_ge.to_csv('daily/ge.csv', index=False)
df_lyg.to_csv('daily/lyg.csv', index=False)
df_shldq.to_csv('daily/shldq.csv', index=False)

### Ingest data
Edit ~/.zipline/extension.py according to zipline docs https://zipline.ml4trading.io/bundles.html#ingesting-data-from-csv-files

In [10]:
!zipline ingest -b stocks-2011-2021

[2021-11-08 02:36:32.716123] INFO: zipline.data.bundles.core: Ingesting shldq-2020.
Loading custom pricing data:   [------------------------------------]    0% | aapl: sid 0
Loading custom pricing data:   [####--------------------------------]   11% | ge: sid 1
Loading custom pricing data:   [########----------------------------]   22% | gme: sid 2
Loading custom pricing data:   [############------------------------]   33% | gs: sid 3
Loading custom pricing data:   [################--------------------]   44% | hsbc: sid 4
Loading custom pricing data:   [####################----------------]   55% | lyg: sid 5
Loading custom pricing data:   [########################------------]   66% | nflx: sid 6
Loading custom pricing data:   [############################--------]   77% | shldq: sid 7
Loading custom pricing data:   [################################----]   88% | tsla: sid 8
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [#####

In [31]:
!zipline bundles

csvdir <no ingestions>
quandl 2021-10-29 00:52:00.736804
quandl 2021-10-29 00:42:42.964339
quandl 2021-10-28 00:23:28.895431
quantopian-quandl <no ingestions>
stocks-2011-2021 2021-11-11 01:21:14.935265


## Backtesting Trading Strategies
Code from https://towardsdatascience.com/algorithmic-trading-based-on-technical-analysis-in-python-80d445dc6943

### Buy and Hold

In [34]:
%%zipline --start 2011-1-1 --end 2011-12-31 --capital-base 10000.0 -o tsla_2011_bah.pkl --no-benchmark

# imports
from zipline.api import order_percent, symbol, record
from zipline.finance import commission

# parameters
SELECTED_STOCK = 'TSLA'

def initialize(context):
    context.asset = symbol(SELECTED_STOCK)
    context.has_ordered = False  
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))

def handle_data(context, data):
    
    # trading logic
    if not context.has_ordered:
        order_percent(context.asset, 1)
        context.has_ordered = True
        
    record(price=data.current(context.asset, 'price'))

,period_open,period_close,capital_used,ending_cash,longs_count,shorts_count,gross_leverage,ending_value,transactions,starting_exposure,...,treasury_period_return,sharpe,period_label,max_leverage,benchmark_period_return,benchmark_volatility,sortino,excess_return,algo_volatility,algorithm_period_return
2011-01-03 21:00:00+00:00,2011-01-03 14:31:00+00:00,2011-01-03 21:00:00+00:00,0.000000,10000.000000,0,0,0.000000,0.00,[],0.00,...,0.0,NaN,2011-01,0.000000,0.0,NaN,NaN,0.0,NaN,0.000000
2011-01-04 21:00:00+00:00,2011-01-04 14:31:00+00:00,2011-01-04 21:00:00+00:00,-10006.250625,-6.250625,1,0,1.000625,10001.25,"[{'amount': 375, 'dt': 2011-01-04 21:00:00+00:...",0.00,...,0.0,-11.224972,2011-01,1.000625,0.0,0.0,-11.224972,0.0,0.005613,-0.000500
2011-01-05 21:00:00+00:00,2011-01-05 14:31:00+00:00,2011-01-05 21:00:00+00:00,0.000000,-6.250625,1,0,1.000622,10061.25,[],10001.25,...,0.0,8.046455,2011-01,1.000625,0.0,0.0,100.857938,0.0,0.057447,0.005500
2011-01-06 21:00:00+00:00,2011-01-06 14:31:00+00:00,2011-01-06 21:00:00+00:00,0.000000,-6.250625,1,0,1.000598,10455.00,[],10061.25,...,0.0,9.380681,2011-01,1.000625,0.0,0.0,708.907608,0.0,0.299951,0.044875
2011-01-07 21:00:00+00:00,2011-01-07 14:31:00+00:00,2011-01-07 21:00:00+00:00,0.000000,-6.250625,1,0,1.000591,10590.00,[],10455.00,...,0.0,11.159479,2011-01,1.000625,0.0,0.0,817.492047,0.0,0.260063,0.058375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 21:00:00+00:00,2011-12-23 14:31:00+00:00,2011-12-23 21:00:00+00:00,0.000000,-6.250625,1,0,1.000598,10462.50,[],10413.75,...,0.0,0.340655,2011-12,1.000764,0.0,0.0,0.528055,0.0,0.514362,0.045625
2011-12-27 21:00:00+00:00,2011-12-27 14:31:00+00:00,2011-12-27 21:00:00+00:00,0.000000,-6.250625,1,0,1.000584,10713.75,[],10462.50,...,0.0,0.386943,2011-12,1.000764,0.0,0.0,0.600428,0.0,0.513860,0.070750
2011-12-28 21:00:00+00:00,2011-12-28 14:31:00+00:00,2011-12-28 21:00:00+00:00,0.000000,-6.250625,1,0,1.000585,10691.25,[],10713.75,...,0.0,0.382035,2011-12,1.000764,0.0,0.0,0.592805,0.0,0.512836,0.068500
2011-12-29 21:00:00+00:00,2011-12-29 14:31:00+00:00,2011-12-29 21:00:00+00:00,0.000000,-6.250625,1,0,1.000581,10773.75,[],10691.25,...,0.0,0.396385,2011-12,1.000764,0.0,0.0,0.615125,0.0,0.511856,0.076750


### Simple Moving Average


In [35]:
%%zipline --start 2011-1-1 --end 2011-12-31 --capital-base 10000.0 -o tsla_2011_sma.pkl --no-benchmark

# imports 
from zipline.api import order_percent, record, symbol, order_target
from zipline.finance import commission

# parameters 
MA_PERIODS = 20
SELECTED_STOCK = 'TSLA'

def initialize(context):
    context.time = 0
    context.asset = symbol(SELECTED_STOCK)
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.has_position = False

def handle_data(context, data):
    context.time += 1
    record(time=context.time)
    if context.time < MA_PERIODS:
        return

    price_history = data.history(context.asset, fields="price", bar_count=MA_PERIODS, frequency="1d")
    ma = price_history.mean()
    
    # cross up
    if (price_history[-2] < ma) & (price_history[-1] > ma) & (not context.has_position):
        order_percent(context.asset, 1.0)
        context.has_position = True
    # cross down
    elif (price_history[-2] > ma) & (price_history[-1] < ma) & (context.has_position):
        order_target(context.asset, 0)
        context.has_position = False

    record(price=data.current(context.asset, 'price'),
           moving_average=ma)

/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


,period_open,period_close,long_exposure,positions,ending_exposure,starting_exposure,long_value,starting_cash,orders,portfolio_value,...,beta,sharpe,sortino,max_drawdown,trading_days,max_leverage,algorithm_period_return,treasury_period_return,price,moving_average
2011-01-03 21:00:00+00:00,2011-01-03 14:31:00+00:00,2011-01-03 21:00:00+00:00,0.0,[],0.0,0.0,0.0,10000.00000,[],10000.00000,...,None,NaN,NaN,0.000000,1,0.000000,0.000000,0.0,NaN,NaN
2011-01-04 21:00:00+00:00,2011-01-04 14:31:00+00:00,2011-01-04 21:00:00+00:00,0.0,[],0.0,0.0,0.0,10000.00000,[],10000.00000,...,None,NaN,NaN,0.000000,2,0.000000,0.000000,0.0,NaN,NaN
2011-01-05 21:00:00+00:00,2011-01-05 14:31:00+00:00,2011-01-05 21:00:00+00:00,0.0,[],0.0,0.0,0.0,10000.00000,[],10000.00000,...,None,NaN,NaN,0.000000,3,0.000000,0.000000,0.0,NaN,NaN
2011-01-06 21:00:00+00:00,2011-01-06 14:31:00+00:00,2011-01-06 21:00:00+00:00,0.0,[],0.0,0.0,0.0,10000.00000,[],10000.00000,...,None,NaN,NaN,0.000000,4,0.000000,0.000000,0.0,NaN,NaN
2011-01-07 21:00:00+00:00,2011-01-07 14:31:00+00:00,2011-01-07 21:00:00+00:00,0.0,[],0.0,0.0,0.0,10000.00000,[],10000.00000,...,None,NaN,NaN,0.000000,5,0.000000,0.000000,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 21:00:00+00:00,2011-12-23 14:31:00+00:00,2011-12-23 21:00:00+00:00,0.0,[],0.0,0.0,0.0,8421.53388,[],8421.53388,...,None,-0.276174,-0.431616,-0.353742,248,1.062235,-0.157847,0.0,27.90,30.6130
2011-12-27 21:00:00+00:00,2011-12-27 14:31:00+00:00,2011-12-27 21:00:00+00:00,0.0,[],0.0,0.0,0.0,8421.53388,[],8421.53388,...,None,-0.275621,-0.430748,-0.353742,249,1.062235,-0.157847,0.0,28.57,30.4135
2011-12-28 21:00:00+00:00,2011-12-28 14:31:00+00:00,2011-12-28 21:00:00+00:00,0.0,[],0.0,0.0,0.0,8421.53388,[],8421.53388,...,None,-0.275071,-0.429886,-0.353742,250,1.062235,-0.157847,0.0,28.51,30.2515
2011-12-29 21:00:00+00:00,2011-12-29 14:31:00+00:00,2011-12-29 21:00:00+00:00,0.0,[],0.0,0.0,0.0,8421.53388,[],8421.53388,...,None,-0.274525,-0.429029,-0.353742,251,1.062235,-0.157847,0.0,28.73,30.0510


### Moving Average Crossover

In [36]:
%%zipline --start 2011-1-1 --end 2011-12-31 --capital-base 10000.0 -o tsla_2011_mac.pkl --no-benchmark

# imports 
from zipline.api import order_percent, record, symbol, order_target
from zipline.finance import commission

# parameters 
SELECTED_STOCK = 'TSLA'
SLOW_MA_PERIODS = 100
FAST_MA_PERIODS = 20

def initialize(context):
    context.time = 0
    context.asset = symbol(SELECTED_STOCK)
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.has_position = False
    
def handle_data(context, data):
    context.time += 1
    if context.time < SLOW_MA_PERIODS:
        return

    fast_ma = data.history(context.asset, 'price', bar_count=FAST_MA_PERIODS, frequency="1d").mean()
    slow_ma = data.history(context.asset, 'price', bar_count=SLOW_MA_PERIODS, frequency="1d").mean()

    # Trading logic
    if (fast_ma > slow_ma) & (not context.has_position):
        order_percent(context.asset, 1.0)
        context.has_position = True
    elif (fast_ma < slow_ma) & (context.has_position):
        order_target(context.asset, 0)
        context.has_position = False

    record(price=data.current(context.asset, 'price'),
           fast_ma=fast_ma,
           slow_ma=slow_ma)

/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


,period_open,period_close,long_value,longs_count,ending_exposure,capital_used,starting_cash,portfolio_value,returns,orders,...,treasury_period_return,alpha,beta,benchmark_period_return,benchmark_volatility,algo_volatility,trading_days,price,fast_ma,slow_ma
2011-01-03 21:00:00+00:00,2011-01-03 14:31:00+00:00,2011-01-03 21:00:00+00:00,0.00,0,0.00,0.0,10000.00000,10000.00000,0.000000,[],...,0.0,None,None,0.0,NaN,NaN,1,NaN,NaN,NaN
2011-01-04 21:00:00+00:00,2011-01-04 14:31:00+00:00,2011-01-04 21:00:00+00:00,0.00,0,0.00,0.0,10000.00000,10000.00000,0.000000,[],...,0.0,None,None,0.0,0.0,0.000000,2,NaN,NaN,NaN
2011-01-05 21:00:00+00:00,2011-01-05 14:31:00+00:00,2011-01-05 21:00:00+00:00,0.00,0,0.00,0.0,10000.00000,10000.00000,0.000000,[],...,0.0,None,None,0.0,0.0,0.000000,3,NaN,NaN,NaN
2011-01-06 21:00:00+00:00,2011-01-06 14:31:00+00:00,2011-01-06 21:00:00+00:00,0.00,0,0.00,0.0,10000.00000,10000.00000,0.000000,[],...,0.0,None,None,0.0,0.0,0.000000,4,NaN,NaN,NaN
2011-01-07 21:00:00+00:00,2011-01-07 14:31:00+00:00,2011-01-07 21:00:00+00:00,0.00,0,0.00,0.0,10000.00000,10000.00000,0.000000,[],...,0.0,None,None,0.0,0.0,0.000000,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 21:00:00+00:00,2011-12-23 14:31:00+00:00,2011-12-23 21:00:00+00:00,8844.30,1,8844.30,0.0,-158.14895,8686.15105,0.004767,[],...,0.0,None,None,0.0,0.0,0.325706,248,27.90,30.6130,27.7070
2011-12-27 21:00:00+00:00,2011-12-27 14:31:00+00:00,2011-12-27 21:00:00+00:00,9056.69,1,9056.69,0.0,-158.14895,8898.54105,0.024452,[],...,0.0,None,None,0.0,0.0,0.326005,249,28.57,30.4135,27.7452
2011-12-28 21:00:00+00:00,2011-12-28 14:31:00+00:00,2011-12-28 21:00:00+00:00,9037.67,1,9037.67,0.0,-158.14895,8879.52105,-0.002137,[],...,0.0,None,None,0.0,0.0,0.325355,250,28.51,30.2515,27.7879
2011-12-29 21:00:00+00:00,2011-12-29 14:31:00+00:00,2011-12-29 21:00:00+00:00,9107.41,1,9107.41,0.0,-158.14895,8949.26105,0.007854,[],...,0.0,None,None,0.0,0.0,0.324806,251,28.73,30.0510,27.8388


### MACD

In [37]:
%%zipline --start 2011-1-1 --end 2011-12-31 --capital-base 10000.0 -o tsla_2011_macd.pkl --no-benchmark

# imports ----
from zipline.api import order_target, record, symbol, set_commission, order_percent
import matplotlib.pyplot as plt
import talib as ta
from zipline.finance import commission

# parameters ----
SELECTED_STOCK = 'TSLA'

 #initialize the strategy 
def initialize(context):
    context.time = 0
    context.asset = symbol(SELECTED_STOCK)
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.has_position = False
    
def handle_data(context, data):
    context.time += 1
    if context.time < 34:
        return
    
    price_history = data.history(context.asset, fields="price", bar_count=34, frequency="1d")
    macd, macdsignal, macdhist = ta.MACD(price_history, 12, 26, 9) 
    
    if (macdsignal[-1] < macd[-1]) and (not context.has_position):
        order_percent(context.asset, 1.0)
        context.has_position = True
        
    if (macdsignal[-1] > macd[-1]) and (context.has_position):
        order_target(context.asset, 0)
        context.has_position = False
        
    record(macd =  macd[-1], macdsignal = macdsignal[-1], macdhist = macdhist[-1], price=price_history[-1])

/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


,period_open,period_close,starting_cash,transactions,longs_count,short_value,short_exposure,positions,gross_leverage,ending_exposure,...,period_label,benchmark_period_return,benchmark_volatility,trading_days,sortino,algorithm_period_return,macd,macdsignal,macdhist,price
2011-01-03 21:00:00+00:00,2011-01-03 14:31:00+00:00,2011-01-03 21:00:00+00:00,10000.000000,[],0,0.0,0.0,[],0.0,0.0,...,2011-01,0.0,NaN,1,NaN,0.000000,NaN,NaN,NaN,NaN
2011-01-04 21:00:00+00:00,2011-01-04 14:31:00+00:00,2011-01-04 21:00:00+00:00,10000.000000,[],0,0.0,0.0,[],0.0,0.0,...,2011-01,0.0,0.0,2,NaN,0.000000,NaN,NaN,NaN,NaN
2011-01-05 21:00:00+00:00,2011-01-05 14:31:00+00:00,2011-01-05 21:00:00+00:00,10000.000000,[],0,0.0,0.0,[],0.0,0.0,...,2011-01,0.0,0.0,3,NaN,0.000000,NaN,NaN,NaN,NaN
2011-01-06 21:00:00+00:00,2011-01-06 14:31:00+00:00,2011-01-06 21:00:00+00:00,10000.000000,[],0,0.0,0.0,[],0.0,0.0,...,2011-01,0.0,0.0,4,NaN,0.000000,NaN,NaN,NaN,NaN
2011-01-07 21:00:00+00:00,2011-01-07 14:31:00+00:00,2011-01-07 21:00:00+00:00,10000.000000,[],0,0.0,0.0,[],0.0,0.0,...,2011-01,0.0,0.0,5,NaN,0.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 21:00:00+00:00,2011-12-23 14:31:00+00:00,2011-12-23 21:00:00+00:00,6781.833645,[],0,0.0,0.0,[],0.0,0.0,...,2011-12,0.0,0.0,248,-1.597813,-0.321817,-1.278778,-0.844576,-0.434202,27.90
2011-12-27 21:00:00+00:00,2011-12-27 14:31:00+00:00,2011-12-27 21:00:00+00:00,6781.833645,[],0,0.0,0.0,[],0.0,0.0,...,2011-12,0.0,0.0,249,-1.594601,-0.321817,-1.256737,-0.969411,-0.287325,28.57
2011-12-28 21:00:00+00:00,2011-12-28 14:31:00+00:00,2011-12-28 21:00:00+00:00,6781.833645,[],0,0.0,0.0,[],0.0,0.0,...,2011-12,0.0,0.0,250,-1.591409,-0.321817,-1.219827,-1.036778,-0.183050,28.51
2011-12-29 21:00:00+00:00,2011-12-29 14:31:00+00:00,2011-12-29 21:00:00+00:00,6781.833645,[],0,0.0,0.0,[],0.0,0.0,...,2011-12,0.0,0.0,251,-1.588236,-0.321817,-1.214376,-1.164694,-0.049682,28.73


### RSI

In [38]:
%%zipline --start 2011-1-1 --end 2011-12-31 --capital-base 10000.0 -o tsla_2011_rsi.pkl --no-benchmark

# imports ----
from zipline.api import order_target, record, symbol, set_commission, order_percent
import matplotlib.pyplot as plt
import talib as ta
from zipline.finance import commission

# parameters ----
SELECTED_STOCK = 'TSLA'
UPPER = 70
LOWER = 30
RSI_PERIOD = 14

 #initialize the strategy 
def initialize(context):
    context.time = 0
    context.asset = symbol(SELECTED_STOCK)
    context.set_commission(commission.PerShare(cost=0.0, min_trade_cost=0))
    context.has_position = False
    
def handle_data(context, data):
    context.time += 1
    if context.time < RSI_PERIOD + 1:
        return
    
    price_history = data.history(context.asset, fields="price", bar_count=RSI_PERIOD+1, frequency="1d")
    rsi = ta.RSI(price_history, timeperiod=RSI_PERIOD)
    
    if rsi[-1] < LOWER and not context.has_position:
        order_percent(context.asset, 1.0)
        context.has_position = True
      
    if rsi[-1] > UPPER and context.has_position:
        order_target(context.asset, 0)
        context.has_position = False
     
    record(rsi=rsi[-1], price=price_history[-1], time=context.time) 

/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


,period_open,period_close,transactions,net_leverage,shorts_count,orders,long_exposure,positions,ending_value,short_exposure,...,algorithm_period_return,max_drawdown,max_leverage,excess_return,trading_days,treasury_period_return,period_label,rsi,price,time
2011-01-03 21:00:00+00:00,2011-01-03 14:31:00+00:00,2011-01-03 21:00:00+00:00,[],0.000000,0,[],0.00,[],0.00,0.0,...,0.000000,0.000000,0.000000,0.0,1,0.0,2011-01,NaN,NaN,NaN
2011-01-04 21:00:00+00:00,2011-01-04 14:31:00+00:00,2011-01-04 21:00:00+00:00,[],0.000000,0,[],0.00,[],0.00,0.0,...,0.000000,0.000000,0.000000,0.0,2,0.0,2011-01,NaN,NaN,NaN
2011-01-05 21:00:00+00:00,2011-01-05 14:31:00+00:00,2011-01-05 21:00:00+00:00,[],0.000000,0,[],0.00,[],0.00,0.0,...,0.000000,0.000000,0.000000,0.0,3,0.0,2011-01,NaN,NaN,NaN
2011-01-06 21:00:00+00:00,2011-01-06 14:31:00+00:00,2011-01-06 21:00:00+00:00,[],0.000000,0,[],0.00,[],0.00,0.0,...,0.000000,0.000000,0.000000,0.0,4,0.0,2011-01,NaN,NaN,NaN
2011-01-07 21:00:00+00:00,2011-01-07 14:31:00+00:00,2011-01-07 21:00:00+00:00,[],0.000000,0,[],0.00,[],0.00,0.0,...,0.000000,0.000000,0.000000,0.0,5,0.0,2011-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-23 21:00:00+00:00,2011-12-23 14:31:00+00:00,2011-12-23 21:00:00+00:00,[],0.986669,0,[],11773.80,"[{'sid': Equity(2862 [TSLA]), 'amount': 422, '...",11773.80,0.0,...,0.193288,-0.175218,1.069519,0.0,248,0.0,2011-12,13.205418,27.90,248.0
2011-12-27 21:00:00+00:00,2011-12-27 14:31:00+00:00,2011-12-27 21:00:00+00:00,[],0.986977,0,[],12056.54,"[{'sid': Equity(2862 [TSLA]), 'amount': 422, '...",12056.54,0.0,...,0.221562,-0.175218,1.069519,0.0,249,0.0,2011-12,15.308370,28.57,249.0
2011-12-28 21:00:00+00:00,2011-12-28 14:31:00+00:00,2011-12-28 21:00:00+00:00,[],0.986950,0,[],12031.22,"[{'sid': Equity(2862 [TSLA]), 'amount': 422, '...",12031.22,0.0,...,0.219030,-0.175218,1.069519,0.0,250,0.0,2011-12,16.430260,28.51,250.0
2011-12-29 21:00:00+00:00,2011-12-29 14:31:00+00:00,2011-12-29 21:00:00+00:00,[],0.987049,0,[],12124.06,"[{'sid': Equity(2862 [TSLA]), 'amount': 422, '...",12124.06,0.0,...,0.228314,-0.175218,1.069519,0.0,251,0.0,2011-12,29.925651,28.73,251.0


### Plots

In [5]:
# plotting function
def plot(title, results, columns_to_plot):
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=[16, 9])

    # portfolio value
    results.portfolio_value.plot(ax=ax[0])
    ax[0].set_ylabel('portfolio value in $')

    # asset
    results[columns_to_plot].plot(ax=ax[1])
    ax[1].set_ylabel('price in $')

    # mark transactions
    perf_trans = results.loc[[t != [] for t in results.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[[t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax[1].plot(buys.index, results.price.loc[buys.index], '^', markersize=10, color='g', label='buy')
    ax[1].plot(sells.index, results.price.loc[sells.index], 'v', markersize=10, color='r', label='sell')
    ax[1].legend()

    # daily returns
    results.returns.plot(ax=ax[2])
    ax[2].set_ylabel('daily returns')

    fig.suptitle(title, fontsize=16)
    plt.legend()
    plt.show()

    print('Final portfolio value (including cash): ${}'.format(np.round(results.portfolio_value[-1], 2)))
